In [87]:
import torch
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.metrics import roc_auc_score, average_precision_score
import scipy.sparse as sp
import numpy as np
import os
import time
from preprocessing import *
import args
from GVAE_GIN import DeepVGAE
import pandas as pd
from torch_geometric.utils import train_test_split_edges
from torch_geometric.data import Data
from sklearn.metrics import roc_auc_score, average_precision_score
from Pytorch_VGAE import CustomGraphDataset
from torch_geometric.data import Data, DataLoader

In [52]:
A_list=[]
X_list=[]
for i in range(5000):
    A_list.append(pd.read_csv('../VGAE_dataset/raw/A/A_{i}.csv'.format(i=i),header=None).values)
    X_list.append(pd.read_csv('../VGAE_dataset/raw/X/X_{i}.csv'.format(i=i),header=None).values)

In [85]:
def adj_to_edge_index(adj):
    # 如果输入是 numpy.ndarray，先转换为 scipy.sparse 矩阵
    if isinstance(adj, np.ndarray):
        adj = sp.csr_matrix(adj)
    # 转换为 COO 格式
    adj_coo = adj.tocoo()
    # 提取行和列的索引
    row = torch.tensor(adj_coo.row, dtype=torch.long)
    col = torch.tensor(adj_coo.col, dtype=torch.long)
    # 堆叠行和列索引，形成 [2, n] 的边列表
    edge_index = torch.stack([row, col], dim=0)
    return edge_index


def prepare_data(X, A):
    # 转换为 PyG 数据格式
    data_list = []
    for i in range(len(X)):
        x = torch.tensor(X[i], dtype=torch.float)
        edge_index = adj_to_edge_index(A[i])
        data = Data(x=x, edge_index=edge_index)
        
        # 分割边用于训练、验证和测试
        data = train_test_split_edges(data)
        data_list.append(data)

    return data_list

In [99]:
from torch_geometric.data import Data, DataLoader
data_list = prepare_data(X=X_list, A=A_list)

d_list = [data_list[0],data_list[1]]
#data_list[1]:Data(x=[25, 22], val_pos_edge_index=[2, 1], test_pos_edge_index=[2, 2], train_pos_edge_index=[2, 48], train_neg_adj_mask=[25, 25], val_neg_edge_index=[2, 1], test_neg_edge_index=[2, 2])

#data_list[0]:Data(x=[17, 22], val_pos_edge_index=[2, 1], test_pos_edge_index=[2, 2], train_pos_edge_index=[2, 36], train_neg_adj_mask=[17, 17], val_neg_edge_index=[2, 1], test_neg_edge_index=[2, 2])
loader = DataLoader(d_list, batch_size=2, shuffle=True)
for data in loader:
    print(data)

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)
/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 17 but got size 25 for tensor number 1 in the list.

In [95]:
for i in data_list:
    if i.x.shape[1] != 22:
        print(i) 
        
    

In [90]:
loader

In [97]:
data_list[0]

Data(x=[17, 22], val_pos_edge_index=[2, 1], test_pos_edge_index=[2, 2], train_pos_edge_index=[2, 36], train_neg_adj_mask=[17, 17], val_neg_edge_index=[2, 1], test_neg_edge_index=[2, 2])

In [83]:
data_list[2].x.size(1)

22

In [98]:
import torch
from torch_geometric.data import Data, DataLoader

# 示例数据
data_0 = Data(x=torch.randn(17, 22), train_pos_edge_index=torch.randint(0, 17, (2, 36)))  # 17 nodes
data_1 = Data(x=torch.randn(25, 22), train_pos_edge_index=torch.randint(0, 25, (2, 48)))  # 25 nodes
data_list = [data_0, data_1]

loader = DataLoader(data_list, batch_size=2, shuffle=True)

for data in loader:
    print(data)


DataBatch(x=[42, 22], train_pos_edge_index=[2, 84], batch=[42], ptr=[3])


/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [104]:
import numpy as np
import torch
import scipy.sparse as sp
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import train_test_split_edges

def adj_to_edge_index(adj):
    # 如果输入是 numpy.ndarray，先转换为 scipy.sparse 矩阵
    if isinstance(adj, np.ndarray):
        adj = sp.csr_matrix(adj)
    # 转换为 COO 格式
    adj_coo = adj.tocoo()
    # 提取行和列的索引
    row = torch.tensor(adj_coo.row, dtype=torch.long)
    col = torch.tensor(adj_coo.col, dtype=torch.long)
    # 堆叠行和列索引，形成 [2, n] 的边列表
    edge_index = torch.stack([row, col], dim=0)
    return edge_index

def prepare_data(X, A):
    # 转换为 PyG 数据格式
    data_list = []
    for i in range(len(X)):
        x = torch.tensor(X[i], dtype=torch.float)
        edge_index = adj_to_edge_index(A[i])
        data = Data(x=x, edge_index=edge_index)
        
        # 分割边用于训练、验证和测试
        data = train_test_split_edges(data)
        data_list.append(data)

    return data_list

# 示例数据准备
X_list = [np.random.rand(17, 22), np.random.rand(25, 22)]
A_list = [np.random.rand(17, 17), np.random.rand(25, 25)]

# 准备数据
data_list = prepare_data(X=X_list, A=A_list)

# 使用 DataLoader
loader = DataLoader(data_list, shuffle=True)

# 打印加载的数据
for data in loader:
    print(data)


DataBatch(x=[25, 22], val_pos_edge_index=[2, 15], test_pos_edge_index=[2, 30], train_pos_edge_index=[2, 510], train_neg_adj_mask=[25, 25], val_neg_edge_index=[2, 0], test_neg_edge_index=[2, 0], batch=[25], ptr=[2])
DataBatch(x=[17, 22], val_pos_edge_index=[2, 6], test_pos_edge_index=[2, 13], train_pos_edge_index=[2, 234], train_neg_adj_mask=[17, 17], val_neg_edge_index=[2, 0], test_neg_edge_index=[2, 0], batch=[17], ptr=[2])


In [102]:
loader

TypeError: 'DataLoader' object is not subscriptable